In [2]:
import pandas as pd
import itertools
import gene_conversion_tools as gct

## Load BioPlex Raw Data
#### Source: http://wren.hms.harvard.edu/bioplex/data/BioPlex_interactionList_v4.tsv
Downloaded: November 22, 2016  
Last Updated: June 12, 2015   

In [3]:
wd = '/cellar/users/jkhuang/Data/Projects/Network_Analysis/Data/'
BioPlex_Raw = pd.read_csv(wd+'Network_Data_Raw/BioPlex_interactionList_v4.tsv',sep='\t')
print 'Raw edge count in BioPlex:', len(BioPlex_Raw)

Raw edge count in BioPlex: 56553


In [5]:
BioPlex_Raw.head()

,GeneA,GeneB,UniprotA,UniprotB,SymbolA,SymbolB,pW,pNI,pInt
0,100,728378,P00813,A5A3E0,ADA,POTEF,2.380860e-09,0.000332,0.999668
1,100,345651,P00813,Q562R1,ADA,ACTBL2,9.790000e-18,0.211914,0.788086
2,222389,708,Q8N7W2,Q07021,BEND7,C1QBP,2.960000e-17,0.005645,0.994355
3,222389,4038,Q8N7W2,O75096,BEND7,LRP4,3.302990e-10,0.000280,0.999720
4,645121,3312,Q6ZMN8,P11142,CCNI2,HSPA8,2.060000e-16,0.036235,0.963765


#### Parse all Entrez genes in BioPlex

In [23]:
# Extract gene list
BioPlex_Human_Genes = list(set(BioPlex_Raw['GeneA']).union(set(BioPlex_Raw['GeneB'])))
BioPlex_Human_Genes = [str(gene) for gene in BioPlex_Human_Genes]

## Convert Genes

In [24]:
# Construct list of genes to be submitted to MyGene.Info API
query_string, valid_genes, invalid_genes = gct.query_constructor(BioPlex_Human_Genes)

10961 Valid Query Genes
0 Invalid Query Genes


In [25]:
# Set scopes (gene naming systems to search)
scopes = "entrezgene, retired, alias"

# Set fields (systems from which to return gene names from)
fields = "symbol, entrezgene"

In [26]:
# Query MyGene.Info
match_list = gct.query_batch(query_string, scopes=scopes, fields=fields)
print len(match_list), 'Matched query results'

Batch query complete: 9.83 seconds
10961 Matched query results


In [27]:
match_table_trim, query_to_symbol, query_to_entrez = gct.construct_query_map_table(match_list, valid_genes)

Queries without full matching results found: 10

0 Queries with mutliple matches found

Query mapping table/dictionary construction complete: 11.32 seconds


## Construct Converted Network

In [28]:
query_edgelist = BioPlex_Raw[['GeneA','GeneB']].values.tolist()
query_edgelist_fmt = [[str(edge[0]),str(edge[1])] for edge in query_edgelist]

In [29]:
BioPlex_edgelist_symbol = gct.convert_edgelist(query_edgelist_fmt, query_to_symbol)
BioPlex_edgelist_symbol_filt = gct.filter_converted_edgelist(BioPlex_edgelist_symbol)
gct.write_edgelist(BioPlex_edgelist_symbol_filt, wd+'Network_SIFs_Symbol/BioPlex_Symbol.sif')

56553 input edges
3 self-edges removed
108 edges with un-mapped genes removed
6 duplicate edges removed
Edge list filtered: 0.06 seconds
56436 Edges remaining
Edge list saved: 0.16 seconds


In [30]:
BioPlex_edgelist_entrez = gct.convert_edgelist(query_edgelist_fmt, query_to_entrez)
BioPlex_edgelist_entrez_filt = gct.filter_converted_edgelist(BioPlex_edgelist_entrez)
gct.write_edgelist(BioPlex_edgelist_entrez_filt, wd+'Network_SIFs_Entrez/BioPlex_Entrez.sif')

56553 input edges
3 self-edges removed
108 edges with un-mapped genes removed
6 duplicate edges removed
Edge list filtered: 0.05 seconds
56436 Edges remaining
Edge list saved: 0.06 seconds
